In [ ]:
import neuraltda.topology2 as tp2
from ephys import core, events
import pandas as pd
from joblib import Parallel, delayed
import neuraltda.simpComp as sc
import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import pickle
import glob
import os
from ephys import core, events, clust


In [ ]:
# Bin The Data

# Correct/Incorrect
winSizes = [5.0, 10.0, 25.0, 50.0]
segmentInfo = [0, 0] # Sample + Target/Distractor period
#segmentInfo = [2500, 0] # Target+Distractor period
povers = 0.5

blockPath='./'
spikes = core.load_spikes(blockPath)
trials = events.load_trials(blockPath)
fs = core.load_fs(blockPath)
clusters = core.load_clusters(blockPath)

correctTrials = trials[trials['correct']==True]
incorrectTrials = trials[trials['correct']==False]

Parallel(n_jobs=5)(delayed(tp2.do_dag_bin_lazy)(blockPath, spikes, correctTrials, clusters, 
                                           fs, winSize, segmentInfo, cluster_group=['Good', 'MUA'], 
                                           dtOverlap=povers*winSize, comment='correct') for winSize in winSizes)

Parallel(n_jobs=5)(delayed(tp2.do_dag_bin_lazy)(blockPath, spikes, incorrectTrials, clusters, 
                                           fs, winSize, segmentInfo, cluster_group=['Good', 'MUA'], 
                                           dtOverlap=povers*winSize, comment='incorrect') for winSize in winSizes)



In [ ]:
blockPath = './'
winSize = 10.0 #ms
thresh = 13.0
povers = 0.5
cluster_group = ['Good', 'MUA']
widenarrow_threshold = 0.000230 # sw threshold in seconds
segmentInfo = [2500, 0] # Sample/Distractor Period
spikes = core.load_spikes(blockPath)
trials = events.load_trials(blockPath)
fs = core.load_fs(blockPath)

# Get wide/narrow clusters
clusters = core.load_clusters(blockPath)
clusters_list = clusters[clusters.quality.isin(cluster_group)]['cluster'].unique()
(wide, narrow) = clust.get_wide_narrow(blockPath, clusters_list, widenarrow_threshold)

# Get Correct/Incorrect Trials
correctTrials = trials[trials['correct']==True]
incorrectTrials = trials[trials['correct']==False]

# Bin and compute SCG
bfdict = tp2.do_dag_bin_lazy(blockPath, spikes, correctTrials, clusters, fs, winSize,
                                    segmentInfo, cluster_group=['Good', 'MUA'],
                                    dt_overlap=povers*winSize, comment='SD-correct')
bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
print('BDF: {}'.format(bdf))

sa.computeChainGroups(blockPath, bdf, thresh, comment='SD-correct-wide', clusters=wide)

bfdict = tp2.do_dag_bin_lazy(blockPath, spikes, correctTrials, clusters, fs, winSize,
                                    segmentInfo, cluster_group=['Good', 'MUA'],
                                    dt_overlap=povers*winSize, comment='SD-correct')
bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
sa.computeChainGroups(blockPath, bdf, thresh, comment='SD-correct-narrow', clusters=narrow)

bfdict2 = tp2.do_dag_bin_lazy(blockPath, spikes, incorrectTrials, clusters, fs, winSize,
                                    segmentInfo, cluster_group=['Good', 'MUA'],
                                    dt_overlap=povers*winSize, comment='SD-incorrect')
bdf2 = glob.glob(os.path.join(bfdict2['raw'], '*.binned'))[0]
sa.computeChainGroups(blockPath, bdf2, thresh, comment='SD-incorrect-wide', clusters=wide)

bfdict2 = tp2.do_dag_bin_lazy(blockPath, spikes, incorrectTrials, clusters, fs, winSize,
                                    segmentInfo, cluster_group=['Good', 'MUA'],
                                    dt_overlap=povers*winSize, comment='SD-incorrect')
bdf2 = glob.glob(os.path.join(bfdict2['raw'], '*.binned'))[0]
sa.computeChainGroups(blockPath, bdf3, thresh, comment='SD-incorrect-narrow', clusters=narrow)

In [ ]:
winSize = 10.0 #ms
thresh = 13.0
povers = 0.5
cluster_group = ['Good', 'MUA']
widenarrow_threshold = 0.000230 # sw threshold in seconds
segmentInfo = [0, -2500.] # Target period
spikes = core.load_spikes(blockPath)
trials = events.load_trials(blockPath)
fs = core.load_fs(blockPath)

#Get Wide/Narrow Clusters
clusters = core.load_clusters(blockPath)
clusters_list = clusters[clusters.quality.isin(cluster_group)]['cluster'].unique()

(wide, narrow) = clust.get_wide_narrow(blockPath, clusters_list, widenarrow_threshold)

# Get Correct/Incorrect Trials
correctTrials = trials[trials['correct']==True]
incorrectTrials = trials[trials['correct']==False]


# bin and make SCG
bfdict = tp2.do_dag_bin_lazy(blockPath, spikes, correctTrials, clusters, fs, winSize,
                                    segmentInfo, cluster_group=['Good', 'MUA'],
                                    dt_overlap=povers*winSize, comment='Targ-correct')
bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
print(bdf)
sa.computeChainGroups(blockPath, bdf, thresh, comment='Targ-correct-wide', clusters=wide)

bfdict = tp2.do_dag_bin_lazy(blockPath, spikes, correctTrials, clusters, fs, winSize,
                                    segmentInfo, cluster_group=['Good', 'MUA'],
                                    dt_overlap=povers*winSize, comment='Targ-correct')
bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]

print(bdf)
sa.computeChainGroups(blockPath, bdf, thresh, comment='Targ-correct-narrow', clusters=narrow)

bfdict2 = tp2.do_dag_bin_lazy(blockPath, spikes, incorrectTrials, clusters, fs, winSize,
                                    segmentInfo, cluster_group=['Good', 'MUA'],
                                    dt_overlap=povers*winSize, comment='Targ-incorrect')
bdf2 = glob.glob(os.path.join(bfdict2['raw'], '*.binned'))[0]
sa.computeChainGroups(blockPath, bdf2, thresh, comment='Targ-incorrect-wide', clusters=wide)
print(bdf)

bfdict2 = tp2.do_dag_bin_lazy(blockPath, spikes, incorrectTrials, clusters, fs, winSize,
                                    segmentInfo, cluster_group=['Good', 'MUA'],
                                    dt_overlap=povers*winSize, comment='Targ-incorrect')
bdf2 = glob.glob(os.path.join(bfdict2['raw'], '*.binned'))[0]
sa.computeChainGroups(blockPath, bdf2, thresh, comment='Targ-incorrect-narrow', clusters=narrow)
                                                                             65,1          Bot
